# Question 1: Online Shopping Platform

Name this Jupyter Notebook as: <br>
`TASK1_<your name>_<class>_<index number>.ipynb`

An online shopping platform stores its data in a SQL database.

The platform needs to store information about products, customers, and order.

Your task is to perform several operations on the database.

For each of the sub-tasks, add a comment statement at the beginning of the code using the hash symbol '#' to indicate the sub-task the program code belongs to.

## Task 1.1 [Modified]

Write Python code to create a SQLite database named online_shop. Ensure that this database is empty before proceeding

Three comma-separated value files are provided, each containing multiple records to be inserted into specific collections within the database. The files are `PRODUCTS.csv`, `CUSTOMERS.csv` and `ORDERS.csv`.

**PRODUCTS.csv:** <br>
`<product_id>,<name>,<model>,<category>,<price>,<stock>`

**CUSTOMERS.csv:** <br>
`<customer_id>,<name>,<email>,<phone>`

**ORDERS.csv:** <br>
`<order_id>,<customer_id>,<product_id>,<quantity>,<order_date>`


Create tables named products, customers, and orders, and insert the data from each CSV file into the corresponding table.

Save the SQL command in a file named: <br>
`TASK1_<your name>_<class>_<index number>_tables.sql`

<div style="text-align: right">[3]</div>

In [1]:
# Task 1.1
import sqlite3
import csv
# CREATE online_shop.db

# Creation of tables
conn = sqlite3.connect("./Resources/TASK1/online_shop.db")
cursor = conn.cursor()
cursor.execute("""CREATE TABLE IF NOT EXISTS products(
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    model TEXT UNIQUE NOT NULL,
    category TEXT NOT NULL,
    price INTEGER NOT NULL,
    stock INTEGER NOT NULL
)""")

cursor.execute("""CREATE TABLE IF NOT EXISTS customers(
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    phone INTEGER NOT NULL
)""")

cursor.execute("""CREATE TABLE IF NOT EXISTS orders(
    order_id INTEGER,
    customer_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    order_date TEXT NOT NULL,
    PRIMARY KEY (order_id,product_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
)""")

conn.commit()
conn.close()

# Insertion of Data
with open("./Resources/TASK1/PRODUCTS.csv") as file:
    reader = csv.reader(file)
    products = list(reader)

products_cleaned = [(int(p[0]),p[1],p[2],p[3],int(p[4]),int(p[5])) for p in products[1::]]

with open("./Resources/TASK1/CUSTOMERS.csv") as file:
    reader = csv.reader(file)
    customers = list(reader)

customers_cleaned = [(int(c[0]),c[1],c[2],c[3]) for c in customers[1::]]

with open("./Resources/TASK1/ORDERS.csv") as file:
    reader = csv.reader(file)
    orders = list(reader)

orders_cleaned = [(int(o[0]),int(o[1]),int(o[2]),int(o[3]),o[4]) for o in orders[1::]]

conn = sqlite3.connect("./Resources/TASK1/online_shop.db")
cursor = conn.cursor()
cursor.executemany("INSERT INTO products (product_id,name,model,category,price,stock) VALUES (?,?,?,?,?,?)",products_cleaned)
cursor.executemany("INSERT INTO customers (customer_id,name,email,phone) VALUES (?,?,?,?)",customers_cleaned)
cursor.executemany("INSERT INTO orders (order_id,customer_id,product_id,quantity,order_date) VALUES (?,?,?,?,?)",orders_cleaned)
conn.commit()
conn.close()

## Task 1.2
Add to your program code to retrieve and display all products in the Books category from the `products` table.

Display the data in a readable format.

**Alternatively:** <br>
Use the `tabulate` module to display your data in a table.

[Reference](https://pypi.org/project/tabulate "Open PYPI Page")

Pip install the tabulate python module by opening ur terminal and running the following command:

> ```bash 
> pip3 install tabulate
>```

**Example Usage:**

>```python
> from tabulate import tabulate
> data = [
>    ["Alice", 24, "Engineer"],
>    ["Bob", 30, "Designer"],
>    ["Charlie", 22, "Student"]
> ]
> headers = ["Name","Age","Occupation"]
> print(tabulate(data,headers=headers))
>
> """
> Output:
> Name      Age  Occupation
> ------  -----  ----------
> Alice      24  Engineer
> Bob        30  Designer
> Charlie    22  Student
> """
>```


Run your program.

<div style="text-align:right;">[2]</div>

In [ ]:
# Task 1.2
import sqlite3
from tabulate import tabulate

conn = sqlite3.connect("./Resources/TASK1/online_shop.db")
cursor = conn.cursor()
cursor.execute("SELECT * from products WHERE category = 'Books'")
data = cursor.fetchall()
headers = ["Product ID","Name","Model","Category","Price","Stock"]
print(tabulate(data,headers=headers))
conn.close()

  Product ID  Name                          Model            Category      Price    Stock
------------  ----------------------------  ---------------  ----------  -------  -------
          11  Novel: The Great Gatsby       Classic Edition  Books            15       50
          12  Cookbook: The Joy of Cooking  2020 Edition     Books            30       30


## Task 1.3
Add to your program code to find all orders placed by the customer named `Charles`. Assume there is only one customer with this name.

Display the details of these orders from the `orders` table in a readable format.

**Alternatively:** <br>
Import `tabulate` module and utilise it as shown above to represent the data in a readable manner.

<div style="text-align:right;">[4]</div>


In [23]:
# Task 1.3
import sqlite3
from tabulate import tabulate

conn = sqlite3.connect('./Resources/TASK1/online_shop.db')
cursor = conn.cursor()
cursor.execute("SELECT customer_id FROM customers WHERE name = 'Charles'")
id = cursor.fetchone()[0]
cursor.execute("SELECT o.order_id,p.name,p.category,o.quantity,p.price,o.order_date FROM orders AS o JOIN products AS p ON o.product_id = p.product_id WHERE customer_id = ? ",(id,))
order_info = cursor.fetchall()
headers = ["Order Id","Product Name","Category","Quantity","Price","Order Date"]
print(tabulate(order_info,headers=headers))



  Order Id  Product Name    Category           Quantity    Price  Order Date
----------  --------------  ---------------  ----------  -------  ------------
         4  Blender         Home Appliances           1       80  2024-04-22
         4  Smartphone      Electronics               1      800  2024-04-22
         4  Smartwatch      Electronics               1      400  2024-04-22
         6  Smartphone      Electronics               1      800  2024-06-18
         6  Laptop          Electronics               1     1200  2024-06-18
        14  Blender         Home Appliances           2       80  2024-08-15
        14  Vacuum Cleaner  Home Appliances           1      500  2024-08-15


## Task 1.4
Add to your program code to calculate the revenue generated from all orders.
Display the revenue in a tabular format with headers for Order ID and Revenue.

For each order, compute the revenue as the sum of the price of each product multiplied by the quantity ordered.

**Alternatively:** <br>
Import `tabulate` module and utilise it as shown above to represent the data in a readable manner.
Run your program.
<div style="text-align: right">[5]</div>

In [ ]:
# Task 1.4
import sqlite3
from tabulate import tabulate

conn = sqlite3.connect("./Resources/TASK1/online_shop.db")
cursor = conn.cursor()
cursor.execute("SELECT o.order_id, SUM(p.price * o.quantity) AS total_amount FROM orders AS o JOIN products AS p ON o.product_id = p.product_id GROUP BY o.order_id;")
order_info = cursor.fetchall()
headers = ["Order ID","Revenue"]
print(tabulate(order_info,headers=headers))






  Order ID    Revenue
----------  ---------
         1        800
         2       2500
         3        230
         4       1280
         5        350
         6       2000
         7       2180
         8        400
         9       5650
        10       1700
        11       2260
        12       4400
        13       2400
        14        660
        15       2250
